In [25]:
from sklearn import tree #decision tree
from sklearn.ensemble import RandomForestRegressor #random Forest
from sklearn.model_selection import cross_val_score # evaluate our model
from sklearn.model_selection import GridSearchCV # gridsearch to find best hyper parameters
import pandas as pd
import numpy as np

In [26]:
schedule_csv = '2020_2021_Schedule_updated.csv'
stats_csv = '2020_2021_stats_updated.csv'

In [27]:
### Data Exploration
np.random.seed(1)
df = pd.read_csv(stats_csv, header=[0,1])
df.head(2)

,Unnamed: 0_level_0,Unnamed: 1_level_0,Overall,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Conf.,...,Unnamed: 28_level_0,Unnamed: 29_level_0,Unnamed: 30_level_0,Unnamed: 31_level_0,Unnamed: 32_level_0,Unnamed: 33_level_0,Unnamed: 34_level_0,Unnamed: 35_level_0,Unnamed: 36_level_0,Unnamed: 37_level_0
,Rk,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8_level_1,W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,20,17,3,0.850,5.93,-7.32,NaN,9,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0
1,2,Air Force,19,4,15,0.211,-12.72,0.17,NaN,2,...,184.0,264.0,0.697,78.0,442.0,240.0,127.0,39.0,277.0,307.0


In [28]:
### Data Cleaning

### Only want a single header
df = pd.read_csv(stats_csv, header=[1])
df.head(2)

,Rk,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8,W.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,20,17,3,0.850,5.93,-7.32,NaN,9,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0
1,2,Air Force,19,4,15,0.211,-12.72,0.17,NaN,2,...,184.0,264.0,0.697,78.0,442.0,240.0,127.0,39.0,277.0,307.0


In [29]:
#rename w.1 etc
df.rename(columns={"W.1": "Conf W", "W.2": "Home W", "W.3": "Away W"}, inplace=True)
df.rename(columns={"L.1": "Conf L", "L.2": "Home L", "L.3": "Away L"}, inplace=True)
df.head(1)

,Rk,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8,Conf W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,20,17,3,0.85,5.93,-7.32,NaN,9,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0


In [30]:
df.drop(['Unnamed: 8', 'Unnamed: 11','Unnamed: 17','Unnamed: 20','Unnamed: 14','Rk'], axis=1, inplace = True)
df.head(4)

,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,Home W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,20,17,3,0.850,5.93,-7.32,9,1,10,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0
1,Air Force,19,4,15,0.211,-12.72,0.17,2,13,2,...,184.0,264.0,0.697,78.0,442.0,240.0,127.0,39.0,277.0,307.0
2,Akron,15,11,4,0.733,5.79,-1.13,9,3,8,...,194.0,260.0,0.746,160.0,592.0,215.0,75.0,41.0,189.0,270.0
3,Alabama A&M,8,6,2,0.750,-13.23,-15.60,4,2,5,...,111.0,172.0,0.645,96.0,329.0,109.0,51.0,30.0,116.0,149.0


In [31]:
df.columns

Index(['School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'Conf W', 'Conf L',
       'Home W', 'Home L', 'Away W', 'Away L', 'Tm.', 'Opp.', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF'],
      dtype='object')

In [32]:
### Data Exploration of new table
schedule_df_full = pd.read_csv(schedule_csv)
schedule_df_full.drop('Unnamed: 0',axis=1, inplace=True)
schedule_df_full.tail()

,Date,visitor_neutral,PTS,home_neutral,PTS.1
3834,2021-03-02,Tulsa,0,UCF,0
3835,2021-03-02,Vanderbilt,0,LSU,0
3836,2021-03-02,Wake Forest,0,Pitt,0
3837,2021-03-02,Western Michigan,0,Northern Illinois,0
3838,2021-03-02,Xavier,0,Georgetown,0


In [33]:
### Drop unneeded column Date
schedule_df_full.drop(['Date'], axis=1, inplace=True)

In [34]:
schedule_df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3839 entries, 0 to 3838
Data columns (total 4 columns):
visitor_neutral    3839 non-null object
PTS                3839 non-null int64
home_neutral       3839 non-null object
PTS.1              3839 non-null int64
dtypes: int64(2), object(2)
memory usage: 120.0+ KB


In [35]:
schedule_df_full.tail()

,visitor_neutral,PTS,home_neutral,PTS.1
3834,Tulsa,0,UCF,0
3835,Vanderbilt,0,LSU,0
3836,Wake Forest,0,Pitt,0
3837,Western Michigan,0,Northern Illinois,0
3838,Xavier,0,Georgetown,0


In [36]:
## separate played and upcoming games
schedule_df_played = schedule_df_full[schedule_df_full['PTS']!=0]
schedule_df_unplayed = schedule_df_full[schedule_df_full['PTS']==0]

In [37]:
## Combine schedule with team statistics

In [38]:
combine_played = pd.merge(schedule_df_played,df, left_on='visitor_neutral', right_on='School')
combine_unplayed = pd.merge(schedule_df_unplayed,df, left_on='visitor_neutral', right_on='School')


In [39]:
Schedule_and_stats_played = pd.merge(combine_played, df, left_on = 'home_neutral', right_on = 'School')
Schedule_and_stats_unplayed = pd.merge(combine_unplayed, df, left_on = 'home_neutral', right_on = 'School')


In [40]:
X = Schedule_and_stats_played.drop(["visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1)
Y = Schedule_and_stats_played[["PTS","PTS.1"]]

In [41]:
reg = tree.DecisionTreeRegressor(random_state=0)
reg = reg.fit(X, Y)

In [42]:
### cross validations
print(cross_val_score(reg, X, Y, cv=5))

#First model performs HORRIBLY!

[-0.47418682 -0.37521486 -0.51108279 -0.61476814 -0.53197773]


In [43]:
# use grid search to determine decision tree depth
params = {"max_depth":range(1,10)}
treereg = tree.DecisionTreeRegressor(random_state=0)
reg2 = GridSearchCV(estimator = treereg, param_grid=params)
reg2.fit(X,Y)
reg2.best_params_


{'max_depth': 4}

In [44]:
means = reg2.cv_results_['mean_test_score']
stds = reg2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, reg2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()


0.027 (+/-0.041) for {'max_depth': 1}

0.085 (+/-0.056) for {'max_depth': 2}

0.101 (+/-0.073) for {'max_depth': 3}

0.103 (+/-0.057) for {'max_depth': 4}

0.089 (+/-0.103) for {'max_depth': 5}

0.033 (+/-0.118) for {'max_depth': 6}

-0.051 (+/-0.108) for {'max_depth': 7}

-0.143 (+/-0.113) for {'max_depth': 8}

-0.239 (+/-0.147) for {'max_depth': 9}



In [45]:
reg3 = RandomForestRegressor(random_state=0)
reg3.fit(X,Y)
print(cross_val_score(reg3, X, Y, cv=5))

[0.2450185  0.26944174 0.17285209 0.21788786 0.20633669]


In [46]:
#get game to predict.  Gonzaga vs Northwestern state
game = Schedule_and_stats_unplayed.drop(["visitor_neutral","home_neutral","School_x","School_y","PTS","PTS.1"], axis=1).head(1)

In [47]:
with pd.option_context('display.max_columns', None):
  display(Schedule_and_stats_unplayed.head(1))

,visitor_neutral,PTS,home_neutral,PTS.1,School_x,G_x,W_x,L_x,W-L%_x,SRS_x,SOS_x,Conf W_x,Conf L_x,Home W_x,Home L_x,Away W_x,Away L_x,Tm._x,Opp._x,MP_x,FG_x,FGA_x,FG%_x,3P_x,3PA_x,3P%_x,FT_x,FTA_x,FT%_x,ORB_x,TRB_x,AST_x,STL_x,BLK_x,TOV_x,PF_x,School_y,G_y,W_y,L_y,W-L%_y,SRS_y,SOS_y,Conf W_y,Conf L_y,Home W_y,Home L_y,Away W_y,Away L_y,Tm._y,Opp._y,MP_y,FG_y,FGA_y,FG%_y,3P_y,3PA_y,3P%_y,FT_y,FTA_y,FT%_y,ORB_y,TRB_y,AST_y,STL_y,BLK_y,TOV_y,PF_y
0,Illinois,0,Nebraska,0,Illinois,18,13,5,0.722,24.8,11.8,9,3,9,2,4,2,1473.0,1239.0,725.0,530.0,1054.0,0.503,134.0,341.0,0.393,279.0,399.0,0.699,190.0,729.0,301.0,100.0,51.0,243.0,324.0,Nebraska,15,4,11,0.267,4.61,10.25,0,8,4,6,0,5,1065.0,1098.0,600.0,369.0,903.0,0.409,115.0,372.0,0.309,212.0,336.0,0.631,158.0,576.0,193.0,113.0,51.0,204.0,272.0


In [48]:
reg3.predict(game)

array([[79.26083333, 65.25333333]])